### Pregunta 1
En la solución de esta pregunta me basé en los algoritmos vistos en las clases de hash criptográfico, para esto hago 3 funciones, __davies_meyer__ la cual genera una función de compresión, __pad__ la cual le agrega un *padding* a un mensaje para que calce con el largo del bloque, y __merkle_damgard__ la cual genera una función de hash basada en las dos funciones anteriores.

In [ ]:
def davies_meyer(encrypt, l_key: int , l_message: int):
    """
    Arguments :
    encrypt : an encryption function
    l_key : length in bytes of the keys for encrypt
    l_message : length in bytes of the messages for encrypt
    Returns :
    A compression function from messages of length l_key + l_message to
    messages of length l_message , defined by using the Davies - Meyer
    construction
    """
    def compress(message):
        part1 = message[:l_key] # parte del input para usar de llave
        part2 = message[len(message)-l_message:] # parte del input para usar de mensaje
        enc = encrypt(part1, part2)
        return bytearray([a ^ b for a, b in zip(enc, part2)]) # se hace un XOR uno a uno entre el mensaje encriptado y part2
    return compress

In [ ]:
def pad(message: bytearray, l_block: int) -> bytearray:
    """
    Arguments :
    message : message to be padded
    l_block : length in bytes of the block
    Returns :
    extension of message that includes the length of message
    (in bytes ) in its last block
    """
    largo_original = len(message) # largo original para después ponerlo en el bloque final
    if len(message) % l_block != 0: # si el largo del mensaje no es divisible por el largo del bloque, se deben agregar un 1 y ceros para que se vuelva divisible
        ceros = l_block - (len(message) % l_block) - 1
        message = message + bytearray([1]) + bytearray(ceros)
    
    message = message + (largo_original % 2**l_block).to_bytes(l_block, "big") # se agrega un bloque al final con el largo del mensaje original.
    return message

In [ ]:
def merkle_damgard(IV: bytearray, comp, l_block: int):
    """
    Arguments :
    IV: initialization vector for a hash function
    comp : compression function to be used in the Merkle - Damgard
    construction
    l_block : length in bytes of the blocks to be used in the Merkle - Damgard
    construction
    Returns :
    A hash function for messages of arbitrary length , defined by using
    the Merkle - Damgard construction
    """
    def hash_making(message):
        cont = 0
        message = pad(message, l_block) # se le agrega el padding al mensaje original
        while cont < len(message):
            chunk = message[cont:cont+l_block] # parte del mensaje de tamaño del largo del bloque
            if cont == 0:
                hashed = comp(chunk + IV) # primer paso donde se usa H0 para la función de compresión.
            else:
                hashed = comp(chunk + hashed) # se comprime el bloque usando como Hn el resultado de hash del paso anterior
            
            cont += l_block
        
        return hashed      
            
    return hash_making
        